In [18]:
import sys
from pathlib import Path
import json
from collections import OrderedDict

import folium as fl
from json2html import json2html as jh
import shapely.geometry as sg
import shapely.ops as so

sys.path.append('../')

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = Path('../data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,[Department of Transport and Main Roads - Tran...
1,timezone,Australia/Brisbane
2,start_date,20140526
3,end_date,20141228
4,num_routes,22
5,num_trips,1339
6,num_stops,416
7,num_shapes,54
8,sample_date,20140529
9,num_routes_active_on_sample_date,20


In [52]:
COLORS_10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
COLORS_SET2 = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f','#e5c494','#b3b3b3']

def make_html(prop):
    return jh.convert(prop, table_attributes="class=\"table table-condensed table-hover\"")
    
def map_routes(feed, route_short_names=None, route_ids=None, date=None, color_palette=COLORS_SET2,
  *, include_stops=True):
    """
    """
    if route_ids is not None:
        cond = lambda x: x['route_id'].isin(route_ids)
    elif route_short_names is not None:
        cond = lambda x: x['route_short_name'].isin(route_short_names)
    else:
        raise ValueError("One of route_ids or route_short_names must be given")
        
    # Get routes DataFrame slice and convert to dictionary
    routes = feed.routes.loc[cond].fillna('n/a').to_dict(orient='records')

    # Create route colors
    n = len(routes)
    colors = [color_palette[i % len(color_palette)] for i in range(n)]

    # Initialize map
    my_map = fl.Map(tiles='cartodbpositron')
    
    # Collect route bounding boxes to set map zoom later
    bboxes = []

    # Create a feature group for each route and add it to the map
    for i, route in enumerate(routes):
        collection = feed.route_to_geojson(
          route_id=route['route_id'], date=date, include_stops=include_stops)
        group = fl.FeatureGroup(name='Route ' + route['route_short_name'])
        color = colors[i]
        
        for f in collection['features']:
            prop = f['properties']
            
            # Add stop
            if f['geometry']['type'] == 'Point':
                lon, lat = f['geometry']['coordinates']
                fl.CircleMarker(
                    location=[lat, lon],
                    radius=8,
                    fill=True,
                    color=color,
                    weight=1,
                    popup=fl.Popup(make_html(prop))
                ).add_to(group)
            
            # Add path
            else:
                prop['color'] = color
                path = fl.GeoJson(f, 
                    name=route,
                    style_function=lambda x: {'color': x['properties']['color']},
                )
                path.add_child(fl.Popup(make_html(prop)))
                path.add_to(group)
                bboxes.append(sg.box(*sg.shape(f['geometry']).bounds))                
                
        group.add_to(my_map)
        
    fl.LayerControl().add_to(my_map)
    
    # Fit map to bounds
    bounds = so.unary_union(bboxes).bounds
    bounds2 = [bounds[1::-1], bounds[3:1:-1]]  # Folium expects this ordering
    my_map.fit_bounds(bounds2)
    
    return my_map



In [56]:
rsns = feed.routes.route_short_name.tolist()[:2]
%time map_routes(feed, rsns, include_stops=False)


CPU times: user 219 ms, sys: 3.99 ms, total: 223 ms
Wall time: 222 ms
